# A sample for Tensorial Convolutional Neural Network

By replacing convolutional kernel with tensor cores, tensorial CNN is constructed.

Here is an tensor ring example to use a TR-based model with `tednet`.

In [1]:
from managpu import GpuManager
my_gpu = GpuManager()
my_gpu.set_by_memory(1)

import random

import tednet as tdt
import tednet.tnn.tensor_ring as tr

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

No GPU Util Limit!
Sorted by memory:
    GPU Index: 1       GPU FreeMemory: 11176 MB       GPU Util: 0%    
    GPU Index: 2       GPU FreeMemory: 11176 MB       GPU Util: 0%    
    GPU Index: 4       GPU FreeMemory: 11176 MB       GPU Util: 0%    
    GPU Index: 0       GPU FreeMemory: 6133 MB        GPU Util: 74%   
    GPU Index: 3       GPU FreeMemory: 1109 MB        GPU Util: 100%  
    GPU Index: 5       GPU FreeMemory: 1109 MB        GPU Util: 100%  
    GPU Index: 6       GPU FreeMemory: 1109 MB        GPU Util: 100%  
    GPU Index: 7       GPU FreeMemory: 1109 MB        GPU Util: 0%    
Qualified GPU Index is: [1]


**Set basic environment**

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
seed = 233
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if use_cuda:
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True

**Set dataloader**

In [3]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=128, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=256, shuffle=True, **kwargs)

**Set training and testing process**

In [4]:
def train(model, device, train_loader, optimizer, epoch, log_interval=200):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

**Begin training**

In [5]:
# Define a TR-LeNet5
model = tr.TRLeNet5(10, [6, 6, 6, 6])
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=2e-2, momentum=0.9, weight_decay=5e-4)

for epoch in range(20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

compression_ration is:  0.3968253968253968
compression_ration is:  14.17233560090703
compression_ration is:  241.54589371980677
compression_ration is:  2.867383512544803
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.633792
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.109367
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.133933

Test set: Average loss: 0.0756, Accuracy: 9751/10000 (98%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.074946
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.039371
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.029103

Test set: Average loss: 0.0691, Accuracy: 9782/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.113578
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.099431
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.084437

Test set: Average loss: 0.0544, Accuracy: 9826/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.130137
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.083295
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.021406

Test set: Average loss: 0.0608, Accuracy: 9799/10000 (98%)

Train 